In [1]:
import pandas as pd
import numpy as np

In [2]:
!rm -f /home/spinster/bootcamp/homework/db/apache.sqlite

In [3]:
# Import SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [4]:
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Text, Numeric
from sqlalchemy import inspect

In [5]:
#csv to df
access_logs_df = pd.read_csv('/home/spinster/bootcamp/homework/apache/infiles/cleaned_access_logs.csv')
responses_df = pd.read_csv('/home/spinster/bootcamp/homework/apache/infiles/coordinates.csv')


In [6]:
#check dtypes and NaN
access_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30694 entries, 0 to 30693
Data columns (total 10 columns):
Unnamed: 0      30694 non-null int64
ip              30694 non-null object
DateTime        30694 non-null object
TimeZone        30694 non-null object
Method          30694 non-null object
URL             30694 non-null object
ResponseCode    30694 non-null object
BytesSent       30694 non-null object
Referer         30694 non-null object
UserAgent       30694 non-null object
dtypes: int64(1), object(9)
memory usage: 2.3+ MB


In [7]:
access_logs_df.head()

,Unnamed: 0,ip,DateTime,TimeZone,Method,URL,ResponseCode,BytesSent,Referer,UserAgent
0,14,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,220,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1,15,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/PMA/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,16,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/dbadmin/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
3,17,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/pma/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,18,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/db/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...


In [8]:
access_logs_df = access_logs_df.drop(['Unnamed: 0'], axis=1)
#found header rows in data while doing analysis. Removed from DB
access_logs_df = access_logs_df[(access_logs_df.ip != 'Host')]

In [9]:
access_logs_df.head()

,ip,DateTime,TimeZone,Method,URL,ResponseCode,BytesSent,Referer,UserAgent
0,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,220,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/PMA/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/dbadmin/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
3,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/pma/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/db/,404,219,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...


In [10]:
#check dtypes and NaN
responses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7724 entries, 0 to 7723
Data columns (total 4 columns):
Unnamed: 0    7724 non-null int64
ip            7724 non-null object
latitude      7724 non-null float64
longitude     7724 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 241.5+ KB


In [11]:
responses_df = responses_df.drop('Unnamed: 0', axis=1)

In [25]:
responses_df

,ip,latitude,longitude
0,62.170.2.205,37.1500,15.1833
1,108.41.39.187,40.5833,-73.9240
2,216.244.66.194,47.4891,-122.2908
3,113.96.223.207,23.1167,113.2500
4,111.206.36.15,39.9289,116.3883
5,52.25.13.124,45.7788,-119.5290
6,123.125.143.15,39.9289,116.3883
7,212.112.33.42,59.6333,17.0667
8,54.37.85.64,48.8582,2.3387
9,107.170.252.110,37.7312,-122.3826


In [13]:
access_respnses_df = pd.merge(access_logs_df, responses_df, how='left',on='ip')

In [14]:
# create engine
engine = create_engine('sqlite:///apache.sqlite')

In [15]:
#convert dfs to dicts in order to create classes
access_response = access_respnses_df.to_dict(orient='records')

In [16]:
#create classes
Base = declarative_base()

class AccessResponse(Base):
    __tablename__ = 'access_response'
    
    id = Column(Integer, primary_key=True)
    ip = Column(Text)
    DateTime= Column(Text)
    TimeZone = Column(Text)
    Method = Column(Text)
    URL = Column(Text)
    ResponseCode = Column(Text)
    Referer = Column(Text)
    BytesSent = Column(Text)
    UserAgent = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.access_response}"

In [17]:
# create db
Base.metadata.create_all(engine)

In [18]:
# check that it worked.
engine.table_names()

['access_response']

In [19]:
# create connection
conn = engine.connect()

In [20]:
# check that all the info is there
AccessResponse.__table__

Table('access_response', MetaData(bind=None), Column('id', Integer(), table=<access_response>, primary_key=True, nullable=False), Column('ip', Text(), table=<access_response>), Column('DateTime', Text(), table=<access_response>), Column('TimeZone', Text(), table=<access_response>), Column('Method', Text(), table=<access_response>), Column('URL', Text(), table=<access_response>), Column('ResponseCode', Text(), table=<access_response>), Column('Referer', Text(), table=<access_response>), Column('BytesSent', Text(), table=<access_response>), Column('UserAgent', Text(), table=<access_response>), Column('latitude', Float(), table=<access_response>), Column('longitude', Float(), table=<access_response>), schema=None)

In [21]:
#delete tables to avoid duplicates when re-run
conn.execute(AccessResponse.__table__.delete())

In [22]:
# execute connection
conn.execute(AccessResponse.__table__.insert(), access_response)

In [23]:
# check that it worked
engine.execute("select ip,longitude from access_response limit 10").fetchall()


[('62.170.2.205', 15.1833),
 ('62.170.2.205', 15.1833),
 ('62.170.2.205', 15.1833),
 ('62.170.2.205', 15.1833),
 ('62.170.2.205', 15.1833),
 ('108.41.39.187', -73.92399999999998),
 ('74.208.182.37', -75.4083),
 ('46.229.164.102', -77.539),
 ('96.57.208.218', -97.822),
 ('96.57.208.218', -97.822)]

In [24]:
#check what's in there
engine.execute("select * from access_response limit 1").fetchall()


[(1, '62.170.2.205', '2018-04-08 07:19:25', 'GMT-0700', 'HEAD', 'http://198.27.242.168:80/phpmyadmin/', '404', '-', '220', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36', 37.15, 15.1833)]